In [ ]:
#create performance evaluation metrics 
def evaluate_model(model, test_env):
    obs, info = test_env.reset()
    rewards = []
    profits = []
    while True:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = test_env.step(action)
        done = terminated or truncated
        
        rewards.append(reward)
        profits.append(info['total_profit'])
        
        if done:
            break
    
    # Calculate metrics
    total_profit = profits[-1]
    sharpe_ratio = np.mean(rewards) / np.std(rewards) if np.std(rewards) != 0 else 0
    max_drawdown = (max(profits) - min(profits)) / max(profits) if max(profits) != 0 else 0
    win_rate = len([r for r in rewards if r > 0]) / len(rewards) if len(rewards) != 0 else 0
    
    return {
        'total_profit': total_profit,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'win_rate': win_rate}

In [ ]:
def buy_and_hold(test_df):
    initial_price = test_df['Close'].iloc[0]  # Price at the start
    final_price = test_df['Close'].iloc[-1]   # Price at the end
    total_profit = final_price - initial_price  # Profit/Loss
    return total_profit

# Calculate Buy & Hold Profit
buy_hold_profit = buy_and_hold(test_df)
print(f"Buy & Hold Total Profit: {buy_hold_profit:.2f}")
# Buy & Hold Metrics
buy_hold_metrics = {
    'total_profit': buy_hold_profit,
    'sharpe_ratio': 0,  # cannot be calculated
    'max_drawdown': 0,  # cannot be calculated
    'win_rate': 0       # cannot be calculated
}
# not sure about the strategy 
def moving_average_crossover(test_df, short_window=10, long_window=20):
    test_df = test_df.copy()
    test_df['short_ma'] = test_df['Close'].rolling(window=short_window).mean()
    test_df['long_ma'] = test_df['Close'].rolling(window=long_window).mean()
    
    # Generate signals
    test_df['signal'] = 0
    test_df.loc[test_df['short_ma'] > test_df['long_ma'], 'signal'] = 1  # Buy
    test_df.loc[test_df['short_ma'] < test_df['long_ma'], 'signal'] = -1 # Sell
    
    # Calculate daily returns
    test_df['daily_return'] = test_df['Close'].pct_change()
    
    # Calculate strategy returns
    test_df['strategy_return'] = test_df['signal'].shift(1) * test_df['daily_return']
    
    # Calculate cumulative profit
    test_df['cumulative_profit'] = (1 + test_df['strategy_return']).cumprod() - 1
    total_profit = test_df['cumulative_profit'].iloc[-1]
    
    return total_profit, test_df

# Calculate Moving Average Crossover Profit
ma_crossover_profit, ma_crossover_df = moving_average_crossover(test_df)
print(f"Moving Average Crossover Total Profit: {ma_crossover_profit:.2f}")
# Moving Average Crossover Metrics
ma_crossover_metrics = {
    'total_profit': ma_crossover_profit,
    'sharpe_ratio': np.mean(ma_crossover_df['strategy_return']) / np.std(ma_crossover_df['strategy_return']),
    'max_drawdown': (ma_crossover_df['cumulative_profit'].max() - ma_crossover_df['cumulative_profit'].min()) / ma_crossover_df['cumulative_profit'].max(),
    'win_rate': len(ma_crossover_df[ma_crossover_df['strategy_return'] > 0]) / len(ma_crossover_df)
}

In [ ]:
#evaluate models 
test_env_ppo = CustomStocksEnv(df=test_df, window_size=window_size, frame_bound=(window_size, len(test_df)))
ppo_metrics = evaluate_model(final_model, test_env_ppo)
print("PPO Model Metrics:", ppo_metrics)

# Evaluate Recurrent PPO Model
test_env_lstm = CustomStocksEnv(df=test_df, window_size=window_size, frame_bound=(window_size, len(test_df)))
lstm_metrics = evaluate_model(final_lstm_model, test_env_lstm)
print("Recurrent PPO Model Metrics:", lstm_metrics)

# Evaluate A2C Model
test_env_a2c = CustomStocksEnv(df=test_df, window_size=window_size, frame_bound=(window_size, len(test_df)))
a2c_metrics = evaluate_model(final_a2c_model, test_env_a2c)
print("A2C Model Metrics:", a2c_metrics)

# creating a df to compare metrics
comparison_df = pd.DataFrame([
    ppo_metrics,
    lstm_metrics,
    a2c_metrics,
    buy_hold_metrics,
    ma_crossover_metrics
], index=['PPO', 'Recurrent PPO', 'A2C', 'Buy & Hold', 'Moving Average Crossover'])

print("Comparison of Model Metrics:")
print(comparison_df)

In [ ]:
# Plot Total Profit Comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df.index, comparison_df['total_profit'], color=['blue', 'green', 'red', 'orange', 'purple'])
plt.title('Total Profit Comparison')
plt.xlabel('Strategy')
plt.ylabel('Total Profit')
plt.show()

# Plot Sharpe Ratio Comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df.index, comparison_df['sharpe_ratio'], color=['blue', 'green', 'red', 'orange', 'purple'])
plt.title('Sharpe Ratio Comparison')
plt.xlabel('Strategy')
plt.ylabel('Sharpe Ratio')
plt.show()

# Plot Max Drawdown Comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df.index, comparison_df['max_drawdown'], color=['blue', 'green', 'red', 'orange', 'purple'])
plt.title('Max Drawdown Comparison')
plt.xlabel('Strategy')
plt.ylabel('Max Drawdown')
plt.show()

# Plot Win Rate Comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df.index, comparison_df['win_rate'], color=['blue', 'green', 'red', 'orange', 'purple'])
plt.title('Win Rate Comparison')
plt.xlabel('Strategy')
plt.ylabel('Win Rate')
plt.show()